# Document Scanner 

In [48]:
import cv2
import numpy as np

In [49]:
# parameters
width = 840
height = 960
kernel = np.ones((5,5))

In [50]:
def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5),1)
    imgCanny = cv2.Canny(imgBlur,100,100)
    imgDilate = cv2.dilate(imgCanny,kernel,iterations=2)
    imgThres = cv2.erode(imgDilate, kernel, iterations=1)
    return imgThres

In [51]:
def getContour(img):
    biggest = np.array([])
    maxArea = 0
    
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 20000:
            # cv2.drawContours(imgResult, cnt, -1, (255,0,0), 5)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            if area > maxArea and len(approx) == 4:
                maxArea = area
                biggest = approx
    return biggest

In [52]:
def getWarp(img,biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0], [width,0], [0,height], [width,height]])
    try:
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        imgOutput = cv2.warpPerspective(img, matrix, (width,height))

        return imgOutput
    except:
        pass

In [53]:
def reorder(myPoints):
    try:
        myPoints = myPoints.reshape((4,2))
        myPointsNew = np.zeros((4,1,2),np.int32)
        add = myPoints.sum(1)
        #print("add", add)
        myPointsNew[0] = myPoints[np.argmin(add)]
        myPointsNew[3] = myPoints[np.argmax(add)]
        diff = np.diff(myPoints,axis=1)
        myPointsNew[1] = myPoints[np.argmin(diff)]
        myPointsNew[2] = myPoints[np.argmax(diff)]
        #print(myPointsNew)
        return myPointsNew
    except:
        pass

In [54]:
video = cv2.VideoCapture(0)
address = "http://172.20.10.4:8080/video"
video.open(address)
video.set(3,width)
video.set(4,height)
video.set(10,150)

while True:
    check, frame = video.read()
    frame = cv2.resize(frame, (width,height))
    imgResult = frame.copy()
    
    imgThres = preProcessing(frame)
    biggest = getContour(imgThres)
    imgWarpped = getWarp(frame,biggest)
    try:
        cv2.imshow("Result", imgWarpped)
    except:
        cv2.imshow("Result", imgResult)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()